In [6]:
# runs the util notebook so that those functions are available
%run utils.ipynb

In [7]:
X, y, features = load_standardized_beans()
X_train, X_valid, X_test, y_train, y_valid, y_test =  split(X,y)

In [8]:
# Baseline RandomForests Classifier
#using n_estimators as 100
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_valid)
print('accuracy: ', np.mean(y_valid == y_pred))

accuracy:  0.9280205655526992


By Default, the Random Forests Classifier already uses "ovr", so a second version is not presented.

In [9]:
test_classifier(classifier, X, y)

Classifier type: RandomForestClassifier
Classifier paramaters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Classifier accuracy:  0.9231508447240868


As was done with the SVM, we will also be showing the difference when using the non standardized version, with the Random Forests Classifier

In [12]:
# Trying SVM classification with the non-standardized data

X, y, features = load_beans()

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
test_classifier(classifier, X, y)

Classifier type: RandomForestClassifier
Classifier paramaters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Classifier accuracy:  0.923737730379603


In this case, the standardized data has a slightly higher accuracy, if we are looking at 3 significant figures or more. Just as was done with the SVM, we will be sticking with standardized Data for the time being.

In [13]:
X, y, features = load_standardized_beans()

The Baseline accuracy at the moment is **92.8%** using the Random Forests Classifier. At this point, the hyperparameters will be tuned, in order to find the best model for our data.

In [27]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 20, 30]
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Best Hyperparameters: {'max_depth': 20, 'n_estimators': 500}
Best Accuracy: 0.9191770756796472


As you can see above, the best value for the maximum depth was 20, with an n_estimator value of 500. I would like to note that it was not feasible to do more than three values in the parameter grid, nor to add additional parameters such as the minimum samples split value, or the minumum samples leaf value, as doing so would increase the computation time to over an hour.